In [12]:
# Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Step 1: Create a soup object from the home page

In [13]:
url = 'https://pages.git.generalassemb.ly/rldaggie/for-scraping/' # open on your web browser to relate downstream outputs
res = requests.get(url) # get contents by hitting url

In [14]:
res.status_code # to confirm that the request has succeeded

200

In [15]:
res.text # returns html

'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8"/>\n    <meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0"/>\n    <title>Nutrition Information</title>\n    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">\n\n  </head>\n  <body>\n    <header>\n      <section class="container">\n        <nav role="navigation" class="navbar navbar-expand-lg navbar-light bg-light">\n<a class="navbar-brand" href="/">Nutrition Information</a>        </nav>\n      </section>\n    </header>\n    <main role="main" class="container">\n      <br>\n      <div class="alert alert-danger">\n        NOTE: This data is super old and rife with errors. It\'s meant for scraping practice only.\n      </div>\n<table id="restaurants" class="table">\n  <thead>\n    

In [16]:
soup = BeautifulSoup(res.text, 'lxml') # parse html for python
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Nutrition Information</title>
<link crossorigin="anonymous" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" rel="stylesheet"/>
</head>
<body>
<header>
<section class="container">
<nav class="navbar navbar-expand-lg navbar-light bg-light" role="navigation">
<a class="navbar-brand" href="/">Nutrition Information</a> </nav>
</section>
</header>
<main class="container" role="main">
<br/>
<div class="alert alert-danger">
        NOTE: This data is super old and rife with errors. It's meant for scraping practice only.
      </div>
<table class="table" id="restaurants">
<thead>
<tr>
<th>Name</th>
<th></th>
</tr>
</thead>
<tbody>
<tr>
<td>
<a href="restaurants/1.html">A&amp;W Restauran

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and path from the `href`. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'href': 'restaurants/1.html'}, 
    {'name': "Applebee's", 'href': 'restaurants/2.html'},
    ...
]
```

In [17]:
restaurants = []
for row in soup.find('tbody').find_all('tr'):
    dic = {'name': row.find('a').text.strip(), 'href': row.find('a').attrs['href']}
    restaurants.append(dic)
    
restaurants

[{'name': 'A&W Restaurants', 'href': 'restaurants/1.html'},
 {'name': "Applebee's", 'href': 'restaurants/2.html'},
 {'name': "Arby's", 'href': 'restaurants/3.html'},
 {'name': 'Atlanta Bread Company', 'href': 'restaurants/4.html'},
 {'name': "Bojangle's Famous Chicken 'n Biscuits",
  'href': 'restaurants/5.html'},
 {'name': 'Buffalo Wild Wings', 'href': 'restaurants/6.html'},
 {'name': 'Burger King', 'href': 'restaurants/7.html'},
 {'name': "Captain D's", 'href': 'restaurants/8.html'},
 {'name': "Carl's Jr.", 'href': 'restaurants/9.html'},
 {'name': "Charley's Grilled Subs", 'href': 'restaurants/10.html'},
 {'name': 'Chick-fil-A', 'href': 'restaurants/11.html'},
 {'name': "Chili's", 'href': 'restaurants/12.html'},
 {'name': 'Chipotle Mexican Grill', 'href': 'restaurants/13.html'},
 {'name': "Church's", 'href': 'restaurants/14.html'},
 {'name': 'Corner Bakery Cafe', 'href': 'restaurants/15.html'},
 {'name': 'Dairy Queen', 'href': 'restaurants/16.html'},
 {'name': "Denny's", 'href': 'res

### Step 3: Using the `href`, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [18]:
rest_links = []
for row in restaurants:
    href = row['href']
    full_url = f"{url}{href}"
    rest_links.append(full_url)
each_scrapes = []
for link in rest_links:
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'lxml')
    each_scrapes.append(soup)

In [23]:
foods = []
for i in each_scrapes:
    food_in_each_rest = []
    for j in i.find('tbody').find_all('tr'):
        food_in_each_rest.append(j.find_all("td"))
    for k in food_in_each_rest:
        dic = {'calories': k[2].text.strip(), 'carbs': k[4].text.strip(), 'category': k[1].text.strip(), 
               "fat": k[3].text.strip(), "name": k[0].text.strip(), "restaurant": i.find('h3').text.strip()}
        foods.append(dic)
foods

[{'calories': '760',
  'carbs': '45',
  'category': 'Burgers',
  'fat': '45',
  'name': 'Original Bacon Double Cheeseburger',
  'restaurant': 'A&W Restaurants'},
 {'calories': '340',
  'carbs': '26',
  'category': 'Entrees',
  'fat': '20',
  'name': 'Coney (Chili) Dog',
  'restaurant': 'A&W Restaurants'},
 {'calories': '370',
  'carbs': '49',
  'category': 'French Fries',
  'fat': '15',
  'name': 'Chili Fries',
  'restaurant': 'A&W Restaurants'},
 {'calories': '670',
  'carbs': '90',
  'category': 'Shakes',
  'fat': '29',
  'name': 'Strawberry Milkshake (small)',
  'restaurant': 'A&W Restaurants'},
 {'calories': '820',
  'carbs': '150',
  'category': 'Shakes',
  'fat': '18',
  'name': 'A&WÂ® Root Beer Freeze (large)',
  'restaurant': 'A&W Restaurants'},
 {'calories': '340',
  'carbs': '57',
  'category': 'Desserts',
  'fat': '9',
  'name': 'Caramel Sundae',
  'restaurant': 'A&W Restaurants'},
 {'calories': '420',
  'carbs': '86',
  'category': 'Shakes',
  'fat': '6',
  'name': 'Strawbe

In [24]:
len(foods)

5131

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 5,131 rows

In [25]:
df = pd.DataFrame(foods)
df

,calories,carbs,category,fat,name,restaurant
0,760,45,Burgers,45,Original Bacon Double Cheeseburger,A&W Restaurants
1,340,26,Entrees,20,Coney (Chili) Dog,A&W Restaurants
2,370,49,French Fries,15,Chili Fries,A&W Restaurants
3,670,90,Shakes,29,Strawberry Milkshake (small),A&W Restaurants
4,820,150,Shakes,18,A&WÂ® Root Beer Freeze (large),A&W Restaurants
...,...,...,...,...,...,...
5126,200,32,Shakes,5,Jr. Original Chocolate Frostyâ¢,Wendy's
5127,260,25,Wraps,10,Grilled Chicken Go Wrap,Wendy's
5128,670,57,Sandwiches,32,Asiago Ranch Chicken Club,Wendy's
5129,330,30,Wraps,16,Spicy Chicken Go Wrap,Wendy's


### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [26]:
df.to_csv('foods.csv', index=False)